# 과제
트랜스포머의 인코더 활용해서 리뷰 분류하기

*   오늘 실습한 코드를 참고해서 트랜스포머의 인코더(셀프 어텐션)구현
*   18기_OOO_transformer.ipynb 형태로 제출해주세요!
* Q는 5까지 있습니다!



In [1]:
import tensorflow as tf

## 멀티 헤드 어텐션 구현
우선 트랜스포머의 인코더의 첫번째 서브층인 멀티 헤드 어텐션층을 클래스로 구현해 봅시다! transformer.ipynb를 참고하세요! 

In [8]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim ##### = d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0


        self.projection_dim = embedding_dim // num_heads

        


        #Q1. ########### WQ, WK, WV, WO에 해당하는 밀집층을 정의하세요 ################
        self.query_dense = tf.keras.layers.Dense(units=embedding_dim)
        self.key_dense = tf.keras.layers.Dense(units=embedding_dim)
        self.value_dense = tf.keras.layers.Dense(units=embedding_dim)
        self.dense = tf.keras.layers.Dense(units=embedding_dim)

        #######################################################################




    #Q2. ############### scaled_dot_product_attention을 구현하세요. 마스킹은 생략합니다. ###############

    def scaled_dot_product_attention(self, query, key, value):

        # 1. 쿼리와 전치된 키를 곱하기
        matmul_qk = tf.matmul(query, key, transpose_b=True)

        # 2. 스케일링하기(dk의 루트 값으로 나눠주기)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)

        # 3. 소프트맥스 함수 취하기
        attention_weights = tf.nn.softmax(logits, axis=-1)

        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    #############################################################################################



    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        #WQ, WK, WV에 해당하는 밀집층 지나기.
        query = self.query_dense(inputs) 
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # Q3. ############# split_heads 함수를 이용해 헤드를 나눠주세요. ################

        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        #########################################################################


        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value) #scaled dot product attention.
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  


        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim)) #헤드를 연결합니다.
        

        
        # Q4. ###################### WO에 해당하는 밀집층을 지나게 하세요 ################
        
        outputs = self.dense(concat_attention)
        
        #########################################################################

        
        return outputs

## 인코더 클래스 설계하기

멀티 헤드 어텐션에 트랜스포머 인코더의 두번째 서브층인 포지션 와이즈 피드 포워드 신경망을 추가하여 인코더 클래스를 설계해봅시다. transformer.ipynb를 참고하세요!

In [3]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)

        attn_output = self.dropout1(attn_output, training=training)

        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)

        ffn_output = self.dropout2(ffn_output, training=training)

        return self.layernorm2(out1 + ffn_output)

######### Q5. 위의 TransformerBlock 클래스 전체를 읽어보고, def__init__을 참조해서


    def call(self, inputs, training):
        attn_output = self.att(inputs)

        attn_output = self.dropout1(attn_output, training=training)

        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1) 

        ffn_output = self.dropout2(ffn_output, training=training)
        
        return self.layernorm2(out1 + ffn_output)


다음 여섯 줄의 함수에 한 줄 한 줄 주석을 달듯 코드를 설명해 주세요. ex) 층정규화, 서브층 ...

답은 아래에 적어주세요.

---------------


1. attn_output = self.att(inputs)

설명: 멀티 헤드 어텐션 (Q, K, V)


-------

2. attn_output = self.dropout1(attn_output, training=training)

설명: 드롭아웃


-------


3. out1 = self.layernorm1(inputs + attn_output)

설명: 층 정규화 + 잔차 연결

-------



4. ffn_output = self.ffn(out1)

설명: 피드 포워드

-------


5. ffn_output = self.dropout2(ffn_output, training=training)

설명:

-------


6. return self.layernorm2(out1 + ffn_output)

설명:


-------



## 포지션 임베딩


앞서 트랜스포머를 설명할 때는 포지셔널 인코딩을 사용하였지만, 이번에는 위치 정보 자체를 학습을 하도록 하는 포지션 임베딩이라는 방법을 사용합니다.

이는 뒤에서 배우게 될 BERT에서 사용하는 방법이기도 합니다.

포지션 임베딩은 임베딩 층(Embedding layer)를 사용하되, 위치 벡터를 학습하도록 하므로 임베딩 층의 첫번째 인자로 단어 집합의 크기가 아니라 문장의 최대 길이를 넣어줍니다.

### 이 아래부터는 읽어만 보시고 코드 실행만 하세요!

In [4]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.pos_emb = tf.keras.layers.Embedding(max_len, embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

## 데이터 로드, 전처리

In [9]:
vocab_size = 20000  # 빈도수 상위 2만개의 단어만 사용
max_len = 200  # 문장의 최대 길이

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수 : {}'.format(len(X_train)))
print('테스트용 리뷰 개수 : {}'.format(len(X_test)))

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [10]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

## 설계한 트랜스포머를 이용한 리뷰 분류

In [11]:
embedding_dim = 32  # 각 단어의 임베딩 벡터의 차원
num_heads = 2  # 어텐션 헤드의 수
dff = 32  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(2, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [12]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs=2, validation_data=(X_test, y_test))

Epoch 1/2
782/782 [==============================] - 27s 30ms/step - loss: 0.5100 - accuracy: 0.7171 - val_loss: 0.2999 - val_accuracy: 0.8716
Epoch 2/2
782/782 [==============================] - 23s 29ms/step - loss: 0.2054 - accuracy: 0.9239 - val_loss: 0.3080 - val_accuracy: 0.8722


In [13]:
print("테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 6s 8ms/step - loss: 0.3080 - accuracy: 0.8722
테스트 정확도: 0.8722
